# 2.2.2 節

### 準備

In [1]:
(define nil (list))
(define (push x xs)
  (if (null? xs) (list x)
      (cons (car xs) (push x (cdr xs)))))

`leaf?`は木構造において、ノードが終端（葉）であることを意味する。

In [2]:
(define (leaf? t)
  (not (pair? t)))

結果の一覧表示

In [3]:
(define (show-lines . xs)
  (define (iter xs)
    (cond ((null? xs) #t)
          (else (display (car xs))
                (newline)
                (iter (cdr xs)))))
  (iter xs))

お馴染みの算術手続き。

In [4]:
(define (square x) (* x x))

## Ex. 2.24
ポインタ表記

* `|`: `car`
* `->`: `cdr`

``
(* . *) -> (* . /)
 |          |
 1         (* . *) -> (* . /)
            |          |
            2         (* . *) -> (* . /)
                       |          |
                       3          4
``

ツリー表記

``
.  *
  / \
 1   *
    / \
   2   *
      / \
     3   4
``

リスト表記は以下の出力

In [5]:
(list 1 (list 2 (list 3 4)))

(1 (2 (3 4)))

## Ex. 2.25
規則正しく書かれたポインタ図式において、
矢印を下に辿ることは`car`に、矢印を右に辿ることは`cdr`に対応する。

```
a:      d          d
(* . *) -> (* . *) -> (* . *) -> (* . /)  # (1 2 (5 7) 9)
 |          |          |          |
 1          2          | a        9
                       |      d
                      (* . *) -> (* . /)  # (5 7)
                       |          | a
                       5          7
(cadaddr a) = 7
```

``
b:
(* . \)  # ((7))
 | a
(* . \)  # (7)
 | a
 7
(caar b) = 7
``

``
c:      d
(* . *) -> (* . \)  # (1 (2 (3 (4 (5 (6 7))))))
 |          | a    d
 1         (* . *) -> (* . \)  # (2 (3 (4 (5 (6 7)))))
            |          | a    d
            2         (* . *) -> (* . \)  # (3 (4 (5 (6 7))))
                       |          | a    d
                       3         (* . *) -> (* . \)  # (4 (5 (6 7)))
                                  |          | a    d
                                  4         (* . *) -> (* . \)  # (5 (6 7))
                                             |          | a    d
                                             5         (* . *) -> (* . \)  # (6 7)
                                                        |          | a
                                                        6          7
(cadadadadadadr c) = 7
``

In [6]:
(define a (list 1 2 (list 5 7) 9))
(define b (list (list 7)))
(define c (list 1 (list 2 (list 3 (list 4 (list 5 (list 6 7)))))))

(car (cdr (car (cdr (cdr a)))))
(car (car b))
(car (cdr (car (cdr (car (cdr (car (cdr (car (cdr (car (cdr c))))))))))))

7

## Ex. 2.26
それぞれ、（順序対の操作ではなく）リスト操作として解釈すると、
* `append`: リスト`x`とリスト`y`を連結
* `cons`: 要素`x`をリスト`y`の先頭に追加
* `list`: 要素`x`と要素`y`から成るリストを生成

In [7]:
(define x (list 1 2 3))
(define y (list 4 5 6))

(show-lines
 (append x y)
 (cons x y)
 (list x y))

(1 2 3 4 5 6)
((1 2 3) 4 5 6)
((1 2 3) (4 5 6))


#t

## Ex. 2.27
リストの先頭（全体）から走査を開始し、
引数の`cdr`に対し自身を再帰的に適用していくのが、リストに対する**浅い再帰**。
引数は常にリストであるので、終了条件は空リストであるかどうかを確かめればよい。

片や、リストの要素に階層構造を想定して、
引数の`car`と`cdr`それぞれに自身を適用するのが、ツリーに対する**深い再帰**。
引数はリストであるとは限らないので、終了条件は空リストと葉の2パターンが必要（`cond`！）。
他方、返り値の型は確定しているので、プログラムを書くときの手がかりとなる。

In [8]:
(define (deep-reverse xs)
  (cond ((null? xs) nil)
        ((leaf? xs) xs)
        (else (push (deep-reverse (car xs)) (deep-reverse (cdr xs))))))

(deep-reverse (list (list 1 2) (list 3 4)))

((4 3) (2 1))

## Ex. 2.28
前問と同じ深い再帰。返り値は常にリストであることに注意する。

In [9]:
(define (fringe xs)
  (cond ((null? xs) nil)
        ((leaf? xs) (list xs))
        (else (append (fringe (car xs)) (fringe (cdr xs))))))

(define x (list (list 1 2) (list 3 4)))
(fringe (list x x))

(1 2 3 4 1 2 3 4)

## Ex. 2.29
吉田モービル（balanced）

```
.            |
        2====*======3
        |           |
   2====*==1     1==*==1
   |       |    10    10
1==*==1 1==*======3
5     5 15        5
```

### a.
コンストラクタとセレクタ。
重要なのは次の公理が成り立つことである:
$$\verb+(constructor + \dots \verb+(selector+_i \verb+ x)+ \dots \verb+)+ = \verb+x+$$
$$\verb+(selector+_i \verb+ (constructor + \dots \verb+x+_i \dots \verb+))+ = \verb+x+_i$$
（直積と射影の関係）

In [10]:
(define (make-mobile left right) (list left right))
(define (left-branch mobile) (car mobile))
(define (right-branch mobile) (cadr mobile))

(define (make-branch length structure) (list length structure))
(define (branch-length branch) (car branch))
(define (branch-structure branch) (cadr branch))

In [11]:
(define test-mobile
  (make-mobile
   (make-branch 2
                (make-mobile
                 (make-branch 2
                              (make-mobile
                               (make-branch 1 5)
                               (make-branch 1 5)))
                 (make-branch 1
                              (make-mobile
                               (make-branch 1 15)
                               (make-branch 3 5)))))
   (make-branch 3
                (make-mobile
                 (make-branch 1 10)
                 (make-branch 1 10)))))
test-mobile

((2 ((2 ((1 5) (1 5))) (1 ((1 15) (3 5))))) (3 ((1 10) (1 10))))

### b.
モービルとブランチが相互再帰的な構造になっているので、
手続きもそのようにするのが自然である。

`branch-weight`はc.で再利用するため、グローバルに定義する。

In [12]:
(define (total-weight mobile)
  (+ (branch-weight (left-branch mobile))
     (branch-weight (right-branch mobile))))
(define (branch-weight branch)
  (let ((struct (branch-structure branch)))
    (if (number? struct) struct
        (total-weight struct))))

(total-weight test-mobile)

50

なお、リストのような汎用的データ型と違って、
モービルやブランチは「空」であることを想定しなくてよい。

Haskell風に書くと、リストの定義は、

>``data List a = Cons a (List a) | Nil``

と、定義の一部としてnilが含まれているのに対して、
モービルはそうではない。

>``data Mobile = Mobile Branch Branch  
data Branch = EndBranch Float Float | Branch Float Mobile``

※このようなデータ型に対する明示的な記述は、Lispの場合はコードの見えるところになくて、
ユーザーはマニュアル等を参照しなくてはならない。
（さもなくばコンストラクタの定義にあたるか。）

### c.
モービルが**balanced**（平衡）であるとは、（ルートを含んだ）**すべての部分モービルについて**、左右のブランチのトルク（の大きさ）が等しいことである。この条件を見落とした人が多かったようだ。

再帰的に表現すると、これは次の3条件に落とし込める。
1. 左のブランチと右のブランチのトルクが等しい。
2. 左のブランチにモービルがあるならば、それがbalancedである。
3. 右のブランチにモービルがあるならば、それがbalancedである。

したがって、これらを`and`で連結すればよい。

2.と3.は「モービルでないか、balancedである」と同値である（実質条件法）。
同じことを2度確かめることになるので、これは述語として抽象化する。

In [13]:
(define (balanced? mobile)
  (define (torque branch)
    (* (branch-length branch) (branch-weight branch)))
  (define (branch-balanced? branch)
    (let ((struct (branch-structure branch)))
      (or (number? struct) (balanced? struct))))
  (let ((left (left-branch mobile)) (right (right-branch mobile)))
    (and (= (torque left) (torque right))
         (branch-balanced? left)
         (branch-balanced? right))))

(balanced? test-mobile)

#t

### d.
ここまで、データ構造の具体的な実装は、コンストラクタとセレクタ以外の手続きにとっては、抽象の壁の向こうにあった。
そのため、コンストラクタとセレクタの定義だけを、整合的に（件の公理を満たすように）変更すればよく、他の手続きに手を加える必要は一切ない。

In [14]:
(define (make-mobile left right) (cons left right))
(define (left-branch mobile) (car mobile))
(define (right-branch mobile) (cdr mobile))

(define (make-branch length structure) (cons length structure))
(define (branch-length branch) (car branch))
(define (branch-structure branch) (cdr branch))

In [15]:
(define test-mobile
  (make-mobile
   (make-branch 2
                (make-mobile
                 (make-branch 2
                              (make-mobile
                               (make-branch 1 5)
                               (make-branch 1 5)))
                 (make-branch 1
                              (make-mobile
                               (make-branch 1 15)
                               (make-branch 3 5)))))
   (make-branch 3
                (make-mobile
                 (make-branch 1 10)
                 (make-branch 1 10)))))

(balanced? test-mobile)

#t

## Ex. 2.30
Ex. 2.27ですでに行った深い再帰。
`map`を使わない方法と使う方法、2通りで実装する。

In [16]:
(define (square-tree tree)
  (cond ((null? tree) nil)
        ((leaf? tree) (square tree))
        (else (cons (square-tree (car tree)) (square-tree (cdr tree))))))

(square-tree (list 1
                   (list 2 (list 3 4) 5)
                   (list 6 7)))

(1 (4 (9 16) 25) (36 49))

In [17]:
(define (square-tree tree)
  (if (leaf? tree) (square tree)
      (map square-tree tree)))

(square-tree (list 1
                   (list 2 (list 3 4) 5)
                   (list 6 7)))

(1 (4 (9 16) 25) (36 49))

### 補註
`square-tree`の2つの異なる実装は、単なる書き方の違いを超えて、
ツリー構造に対する解釈の違いを反映したものとして捉えることができる。

SICPにおけるツリーの実装は、内部構造としては一意であるが、実は2通りの解釈を許している。
すなわち、片や再起的なペアとして（ポインタ図に対応する）、
片やサブツリーのリストとして（分岐式のツリー図に対応する）。
BN記法を用いてこの違いを表現すると、
前者、後者はそれぞれ次のように定義される。
ちょうどコード上の`cond`節に対応している。）

1. `Tree = Leaf | Nil | (Tree . Tree)`
2. `Tree = Leaf | [Tree]`

1.では`Leaf`の他に、ペアに関する用語である`Nil`や`Cons`（`(* . *)`）がプリミティブとして定義に紛れ込んでいる。
そのため、ツリーの使用者は、`null?`や`car`や`cdr`と言った、ペアの使い方を理解している必要がある。

一方、2.では、プリミティブは`Leaf`と`List`（`[*]`）である。
もちろん、`[Tree] = Nil | (Tree . [Tree])`であるので、これを崩した内部構造は1.と同じであるのだが、
重要なのは、ペアに直接関わる要素を抽象の壁の向こうに追いやることに成功しているということだ。
ツリー使用者は、リストに対する操作法（`map`はその一つ）だけを理解していればよい。

これら1.と2.の違いは、データ構造の定義に依拠する違いであり、
型の概念がないLispにおいて、それらはプログラマの暗黙の了解、あるいはせいぜいドキュメント上の仕様として表現する他ない。
そのため、手続きの定義に複数の方法があったり、1（Leaf）や`Nil`をツリーとして認めるかといった多義性が生じることになる。
強い型付け言語であるHaskellでは、前述のBN記法をほとんどそのままデータ型の定義として用いることができる。
そうした場合、2つのコードのうち片方のみが正しく、片方は違反として排除される。

## Ex. 2.31
前問のどちらかで、`square`の部分を抽象すればよいだけ。

In [18]:
(define (tree-map proc tree)
  (cond ((null? tree) nil)
        ((leaf? tree) (proc tree))
        (else (cons (tree-map proc (car tree)) (tree-map proc (cdr tree))))))
(define (square-tree tree) (tree-map square tree))

(square-tree (list 1
                   (list 2 (list 3 4) 5)
                   (list 6 7)))

(1 (4 (9 16) 25) (36 49))

ただし、後者の`map`式を抽象する場合は、文法上の問題のため、ラムダ式が必要になる。
（Haskellのような部分適用があればよいが。）

In [19]:
(define (tree-map proc tree)
  (if (leaf? tree) (proc tree)
      (map (lambda (sub-tree) (tree-map proc sub-tree)) tree)))
(define (square-tree tree) (tree-map square tree))

(square-tree (list 1
                   (list 2 (list 3 4) 5)
                   (list 6 7)))

(1 (4 (9 16) 25) (36 49))

## Ex. 2.32
空でない集合$S$と、その要素$a$を考える。
$S$の部分集合は、$a$を含まないと含むものに分けられる。
$S$の部分集合で、$a$を含まないものの全体を$\Pi_1$、
$a$を含むものの全体を$\Pi_2$とすると、
$$\mathscr{P}(S) = \Pi_1 + \Pi_2$$
（$+$は直和である。）

ここで、$i_a(X) = X + \{a\}$とすると、$\Pi_1$と$\Pi_2$は$i_a$によって一対一に対応する。
特に、$i_a(\Pi_1) = \Pi_2$である。
また、$T = S - \{a\}$とすれば、$\Pi_1 = \mathscr{P}(T)$であるので、結局、
$$
\mathscr{P}(S) = \mathscr{P}(T) + i_a(\mathscr{P}(T))
$$
あとはこれをコードに落とし込めばよい。
それぞれ、Schemeに翻訳していくと、
* $S$: `s`
* $a$: `(car s)`
* $T = S - \{a\}$: `(cdr s)`
* $\Pi_1 = \mathscr{P}(T)$: `rest = (subsets (cdr s))`
* $i_a(X)$: `(cons (car s) X)`
* $\Pi_2 = i_a(\Pi_1)$: `(map (lambda (subset) (cons (car s) subset)) rest)`

よって、$\mathscr{P}(S)$は……

In [20]:
(define (subsets s)
  (if (null? s) (list nil)
      (let ((rest (subsets (cdr s))))
        (append rest
                (map (lambda (subset) (cons (car s) subset)) rest)))))

(subsets (list 1 2 3 4))

(() (4) (3) (3 4) (2) (2 4) (2 3) (2 3 4) (1) (1 4) (1 3) (1 3 4) (1 2) (1 2 4) (1 2 3) (1 2 3 4))

### 補註
迷った人が多かったようだが、所詮は穴埋めなので、丁寧に推論していけば解けるはずの問題。

    1: (define (subsets s)
    2:   (if (null? s) (list nil)
    3:       (let ((rest (subsets (cdr s))))
    4:         (append rest (map (??) rest)))))

2行目: 空集合の場合の処理。空集合の冪集合は空集合のシングルトン。ベースケースらしく至極当然。

以下、再帰ケースでは具体的な呼び出しとして`s = (1 2 3)`について考える。

3行目: `rest`を`(subsets (cdr s))`とする。
再帰的定義の中で定義中の手続きを用いるときは、**その手続きはすでにきちんと定義されたものとして捉えること**。
今回の場合、`rest`は期待通り`(() (2) (3) (2 3))`に束縛されたものとして考える。

4行目: `(append rest (map (??) rest))`が`(1 2 3 4)`の冪集合、すなわち`(() (1) (2) (3) (1 2) (2 3) (3 1) (1 2 3))`となればよい。
先ほど見たように、`rest`は`(() (2) (3) (2 3))`であるので、`append`の右辺`(map (??) rest)`の部分が`((1) (1 2) (3 1) (1 2 3))`になればよい。

`rest`に`(??)`を`map`して`((1) (1 2) (3 1) (1 2 3))`になる。
あとは具体的に`rest`と目標のリストを見比べれば、`(??)`がどのような手続きでなくてはならないか分かるはずだ。

    rest: (() (2) (3) (2 3))
    (map (??) rest): ((1) (1 2) (3 1) (1 2 3))